In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define years for interpolation
years = [1990, 2019, 2021]

# Country-wise Parkinson's disease deaths (in thousands), estimated
china_deaths = [20, 90, 115]
india_deaths = [10, 40, 50]
europe_deaths = [30, 100, 120]
us_deaths = [15, 110, 130]
germany_deaths = [5, 30, 35]
brazil_deaths = [2, 15, 18]
japan_deaths = [3, 20, 25]
uk_deaths = [3, 20, 22]
russia_deaths = [5, 40, 45]

# Interpolate from 1990 to 2024
full_years = np.arange(1990, 2025, 1)
china_interp = np.interp(full_years, years, china_deaths)
india_interp = np.interp(full_years, years, india_deaths)
europe_interp = np.interp(full_years, years, europe_deaths)
us_interp = np.interp(full_years, years, us_deaths)
germany_interp = np.interp(full_years, years, germany_deaths)
brazil_interp = np.interp(full_years, years, brazil_deaths)
japan_interp = np.interp(full_years, years, japan_deaths)
uk_interp = np.interp(full_years, years, uk_deaths)
russia_interp = np.interp(full_years, years, russia_deaths)

# Plotting
plt.figure(figsize=(12, 8))
plt.plot(full_years, china_interp, label='China', color='green')
plt.plot(full_years, india_interp, label='India', color='red')
plt.plot(full_years, europe_interp, label='Europe', color='purple')
plt.plot(full_years, us_interp, label='US', color='orange')
plt.plot(full_years, germany_interp, label='Germany', color='brown')
plt.plot(full_years, brazil_interp, label='Brazil', color='cyan')
plt.plot(full_years, japan_interp, label='Japan', color='magenta')
plt.plot(full_years, uk_interp, label='UK', color='pink')
plt.plot(full_years, russia_interp, label='Russia', color='yellow')

# Labels and title
plt.xlabel('Year')
plt.ylabel('Deaths (thousands)')
plt.title("Parkinson's Disease Deaths by Country (1990–2024)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile
import sys

# --- Step 1: Repair the PDB File ---
fixer = PDBFixer(filename="5caw.pdb")
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()

# Add hydrogens at a given pH (this may help with aromaticity perception)
fixer.addMissingHydrogens(pH=7.0)

output_pdb = "repaired.pdb"
with open(output_pdb, "w") as f:
    PDBFile.writeFile(fixer.topology, fixer.positions, f)
print("Repaired PDB file written to:", output_pdb)

# --- Step 2: Convert PDB to PDBQT via Command-Line Open Babel ---
import subprocess
try:
    subprocess.run(
        'obabel repaired.pdb -O repaired.pdbqt --addhydrogens --partialcharge gasteiger',
        shell=True,
        check=True
    )
    print("Converted PDBQT file written to: repaired.pdbqt")
except subprocess.CalledProcessError as e:
    print("Error during Open Babel conversion:", e)
    sys.exit(1)


In [ ]:
from rdkit import Chem

# Load molecules from the SDF file.
supplier = Chem.SDMolSupplier("UBR_box.sdf")
if not supplier:
    raise IOError("Could not open compounds.sdf")

# Iterate over each molecule and write it to an individual SDF file.
for idx, mol in enumerate(supplier):
    if mol is None:
        print(f"Skipping molecule at index {idx} (failed to parse)")
        continue
    # Create a writer for the current molecule.
    writer = Chem.SDWriter(f"UBR_box_{idx}.sdf")
    writer.write(mol)
    writer.close()
    print(f"Saved compound_{idx}.sdf")


In [ ]:
#!/usr/bin/env python
import os
import glob
import json
import math
import subprocess
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Lipinski

###############################
# Configuration
###############################
# Folder containing raw training compounds in PDB format
INPUT_FOLDER = "compounds"  

# Receptor file in PDBQT format
PROTEIN_PDBQT = "5cawrepair.pdbqt"

# Output folder: filtered compounds that pass docking criteria
OUTPUT_FOLDER = "training_compounds_filtered"

# Checkpoint file to track processed files
CHECKPOINT_FILE = "docking_training_checkpoint.json"

# Docking parameters
DOCKING_CENTER = [-12.646, 5.094, -59.525]   # Adjust based on your binding site
DOCKING_SIZE = [30, 30, 30]                    # Dimensions of the docking box
AFFINITY_CUTOFF = -7.0                         # Only compounds with docking score < -7.0 are accepted

# Create necessary folders if they don't exist
for folder in [OUTPUT_FOLDER, "temp_files"]:
    if not os.path.exists(folder):
        os.makedirs(folder)
TEMP_FOLDER = "temp_files"

###############################
# Checkpoint Functions
###############################
def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            return json.load(f)
    return {"processed": []}

def save_checkpoint(data):
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(data, f, indent=2)

###############################
# Lipinski Rule of 5 Check
###############################
def passes_lipinski_ro5(mol):
    try:
        mw = Descriptors.MolWt(mol)
        logp = Descriptors.MolLogP(mol)
        hbd = Lipinski.NumHDonors(mol)
        hba = Lipinski.NumHAcceptors(mol)
        return (mw <= 500 and logp <= 5 and hbd <= 5 and hba <= 10)
    except Exception as e:
        print("Error in Lipinski check:", e)
        return False

###############################
# Conversion Function: MOL -> PDBQT
###############################
def convert_to_pdbqt(mol, pdbqt_path):
    temp_pdb = os.path.join(TEMP_FOLDER, "temp.pdb")
    Chem.MolToPDBFile(mol, temp_pdb)
    try:
        subprocess.run(
            ["obabel", temp_pdb, "-O", pdbqt_path, "--partialcharge", "gasteiger"],
            check=True,
            capture_output=True
        )
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error converting to PDBQT for {pdbqt_path}: {e}")
        return False
    finally:
        if os.path.exists(temp_pdb):
            os.remove(temp_pdb)

###############################
# Docking Function using AutoDock Vina
###############################
def run_vina_docking(pdbqt_path):
    output_file = pdbqt_path.replace(".pdbqt", "_out.pdbqt")
    try:
        subprocess.run(
            [
                "vina",
                "--receptor", PROTEIN_PDBQT,
                "--ligand", pdbqt_path,
                "--center_x", str(DOCKING_CENTER[0]),
                "--center_y", str(DOCKING_CENTER[1]),
                "--center_z", str(DOCKING_CENTER[2]),
                "--size_x", str(DOCKING_SIZE[0]),
                "--size_y", str(DOCKING_SIZE[1]),
                "--size_z", str(DOCKING_SIZE[2]),
                "--exhaustiveness", "8",
                "--out", output_file
            ],
            check=True,
            capture_output=True
        )
        with open(output_file, 'r') as f:
            for line in f:
                if "REMARK VINA RESULT" in line:
                    return float(line.split()[3])
        return None
    except Exception as e:
        print(f"Docking failed for {pdbqt_path}: {e}")
        return None

###############################
# Main Workflow: Docking and Filtering
###############################
def main():
    checkpoint = load_checkpoint()
    processed = checkpoint.get("processed", [])
    
    # Get all PDB files from the input folder
    pdb_files = glob.glob(os.path.join(INPUT_FOLDER, "*.pdb"))
    print(f"Found {len(pdb_files)} PDB files in {INPUT_FOLDER}.")
    
    for pdb_file in pdb_files:
        base_name = os.path.basename(pdb_file)
        if base_name in processed:
            continue
        
        print(f"\nProcessing {base_name}...")
        mol = Chem.MolFromPDBFile(pdb_file, removeHs=False)
        if mol is None:
            print(f"Failed to load molecule from {base_name}.")
            processed.append(base_name)
            save_checkpoint({"processed": processed})
            continue
        
        mol = Chem.AddHs(mol)
        if mol.GetNumConformers() == 0:
            AllChem.EmbedMolecule(mol)
            AllChem.UFFOptimizeMolecule(mol)
        
        # Convert molecule to PDBQT
        pdbqt_path = os.path.join(TEMP_FOLDER, f"temp_{base_name}.pdbqt")
        if not convert_to_pdbqt(mol, pdbqt_path):
            print(f"{base_name} failed PDBQT conversion.")
            processed.append(base_name)
            save_checkpoint({"processed": processed})
            continue
        
        # Dock the molecule
        score = run_vina_docking(pdbqt_path)
        if score is not None:
            print(f"{base_name}: docking score = {score}")
            if score < AFFINITY_CUTOFF and passes_lipinski_ro5(mol):
                # Save accepted compound as an individual SDF file named by the original file (without extension)
                out_filename = os.path.join(OUTPUT_FOLDER, f"{os.path.splitext(base_name)[0]}.sdf")
                writer = Chem.SDWriter(out_filename)
                mol.SetProp("Docking_Score", str(score))
                writer.write(mol)
                writer.close()
                print(f"Accepted {base_name} saved to {out_filename}")
            else:
                print(f"{base_name} did not meet filtering criteria.")
        else:
            print(f"{base_name}: docking failed or no score obtained.")
        
        processed.append(base_name)
        save_checkpoint({"processed": processed})
        if os.path.exists(pdbqt_path):
            os.remove(pdbqt_path)
    
    print("\nProcessing complete. Filtered compounds saved in:", OUTPUT_FOLDER)

if __name__ == "__main__":
    main()


In [ ]:
from rdkit import Chem
import pandas as pd

def convert_sdf_to_smiles(sdf_file, output_csv):
    suppl = Chem.SDMolSupplier(sdf_file)
    data = []

    for mol in suppl:
        if mol is not None:
            smiles = Chem.MolToSmiles(mol)
            compound_id = mol.GetProp("_Name") if mol.HasProp("_Name") else "Unknown"
            data.append([compound_id, smiles])

    df = pd.DataFrame(data, columns=["Compound_ID", "SMILES"])
    df.to_csv(output_csv, index=False)
    print(f"Saved {len(df)} compounds to {output_csv}")

convert_sdf_to_smiles("cleanedsolu.sdf", "fragments_smiles.csv")



In [ ]:
from rdkit.Chem import Descriptors

def compute_molecular_features(input_csv, output_csv):
    df = pd.read_csv(input_csv)
    features = []

    for _, row in df.iterrows():
        mol = Chem.MolFromSmiles(row["SMILES"])
        if mol:
            mw = Descriptors.MolWt(mol)
            hbd = Descriptors.NumHDonors(mol)
            hba = Descriptors.NumHAcceptors(mol)
            rb = Descriptors.NumRotatableBonds(mol)
            logp = Descriptors.MolLogP(mol)

            lipinski_pass = (mw <= 500 and hbd <= 5 and hba <= 10 and logp <= 5 and rb <= 10)

            features.append([row["Compound_ID"], row["SMILES"], mw, hbd, hba, rb, logp, lipinski_pass])

    df_features = pd.DataFrame(features, columns=["Compound_ID", "SMILES", "MW", "HBD", "HBA", "RB", "LogP", "Lipinski_Pass"])
    df_features.to_csv(output_csv, index=False)
    print(f"Saved molecular features to {output_csv}")

compute_molecular_features("fragments_smiles.csv", "compound_features_new.csv")



In [ ]:
#!/usr/bin/env python
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

# Configuration: File paths for the two datasets
KNOWN_CSV     = "Enamine.csv"                # Known reported compounds
FRAGMENTS_CSV = "compound_features_new.csv"  # New fragments to compare
OUTPUT_CSV    = "Enamine_matching_results.csv"

def ensure_id_column(df, id_col_name, prefix):
    """If id_col_name is missing, add it as prefix_1, prefix_2, …."""
    if id_col_name not in df.columns:
        df[id_col_name] = [f"{prefix}_{i+1}" for i in range(len(df))]
    return df

def find_and_rename_smiles(df):
    """
    Find any column whose name contains 'smile' (case‐insensitive),
    strip it, and rename that column to exactly 'SMILES'. Returns True if found.
    """
    # strip leading/trailing whitespace
    cols = [c.strip() for c in df.columns]
    df.columns = cols

    # look for any column containing 'smile'
    for c in cols:
        if "smile" in c.lower():
            if c != "SMILES":
                df.rename(columns={c: "SMILES"}, inplace=True)
            return True
    return False

def get_fingerprint(smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)

def main():
    known_df = pd.read_csv(KNOWN_CSV, sep=None, engine='python')
    frag_df  = pd.read_csv(FRAGMENTS_CSV)

    # robustly find & rename the SMILES column
    ok1 = find_and_rename_smiles(known_df)
    ok2 = find_and_rename_smiles(frag_df)
    if not (ok1 and ok2):
        print("Error: couldn’t find a column containing 'SMILES' in one of the files.")
        print("Known CSV columns:", known_df.columns.tolist())
        print("Fragments CSV columns:", frag_df.columns.tolist())
        return

    # ensure ID columns
    known_df = ensure_id_column(known_df, "Catalog ID", "Known")
    frag_df  = ensure_id_column(frag_df, "Fragment_ID", "Frag")

    # compute fingerprints
    print("Computing fingerprints for known compounds...")
    known_df["Fingerprint"] = known_df["SMILES"].map(get_fingerprint)
    print("Computing fingerprints for fragments...")
    frag_df["Fingerprint"]  = frag_df["SMILES"].map(get_fingerprint)

    # drop invalid SMILES
    known_df = known_df[known_df["Fingerprint"].notnull()].reset_index(drop=True)
    frag_df  = frag_df[frag_df["Fingerprint"].notnull()].reset_index(drop=True)

    records = []
    # for each known compound, find best‐matching fragment
    for _, kr in known_df.iterrows():
        k_id     = kr["Catalog ID"]
        k_smiles = kr["SMILES"]
        k_fp     = kr["Fingerprint"]

        best_sim      = -1.0
        best_frag_id  = None
        best_frag_sm  = None

        for _, fr in frag_df.iterrows():
            f_smiles = fr["SMILES"]
            f_fp      = fr["Fingerprint"]
            sim       = DataStructs.TanimotoSimilarity(k_fp, f_fp)
            if sim > best_sim:
                best_sim       = sim
                best_frag_id   = fr["Fragment_ID"]
                best_frag_sm   = f_smiles

        records.append({
            "Known_Compound_ID":       k_id,
            "Known_SMILES":            k_smiles,
            "Matched_Fragment_ID":     best_frag_id,
            "Matched_Fragment_SMILES": best_frag_sm,
            "Matching_Percentage":     best_sim * 100
        })

    out_df = pd.DataFrame(records)
    out_df.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved matching results to {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python
import os
import re
import requests
from rdkit import Chem

# Configuration
INPUT_SDF = "Enamine.sdf"         # Multi-compound SDF file
OUTPUT_DIR = "Enamine_sdfs"  # Folder to save individual SDF files
PUBCHEM_TIMEOUT = 10             # seconds for HTTP requests

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Helper: find the SDF property key that contains 'catalog'
def find_catalog_field(mol):
    for prop_name in mol.GetPropNames():
        if 'catalog' in prop_name.lower():
            return prop_name
    return None

# Helper: sanitize filename
def sanitize_name(name):
    name = name.strip()
    # replace invalid chars
    return re.sub(r"[^\w\-_. ]", "_", name)

# PubChem lookup functions
def fetch_pubchem_title(identifier):
    """Try to fetch Title for a compound by name or CID from PubChem."""
    # Try lookup by name
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{identifier}/property/Title/JSON"
    resp = requests.get(url, timeout=PUBCHEM_TIMEOUT)
    if resp.status_code == 200:
        try:
            return resp.json()['PropertyTable']['Properties'][0]['Title']
        except Exception:
            pass
    # Try lookup by CID
    url2 = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{identifier}/property/Title/JSON"
    resp2 = requests.get(url2, timeout=PUBCHEM_TIMEOUT)
    if resp2.status_code == 200:
        try:
            return resp2.json()['PropertyTable']['Properties'][0]['Title']
        except Exception:
            pass
    return None

# Main splitting logic
supplier = Chem.SDMolSupplier(INPUT_SDF, removeHs=False)
print(f"Loaded {len(supplier)} molecules from {INPUT_SDF}.")

for idx, mol in enumerate(supplier, start=1):
    if mol is None:
        print(f"Skipping molecule #{idx}: parse error.")
        continue

    # Find the catalog field dynamically
    cat_field = find_catalog_field(mol)
    if cat_field:
        identifier = mol.GetProp(cat_field).strip()
    else:
        identifier = f"compound_{idx}"

    # Try to fetch a PubChem title
    pubchem_name = None
    try:
        pubchem_name = fetch_pubchem_title(identifier)
    except Exception as e:
        print(f"Warning: PubChem lookup failed for '{identifier}': {e}")

    # Choose the filename
    base = pubchem_name or identifier
    base = sanitize_name(base)
    out_path = os.path.join(OUTPUT_DIR, f"{base}.sdf")

    # Write the molecule
    writer = Chem.SDWriter(out_path)
    writer.write(mol)
    writer.close()
    print(f"Saved molecule #{idx} as '{out_path}'")


In [ ]:
from rdkit import Chem

def check_molecule(path, name):
    mol = Chem.MolFromMolFile(path)
    print(f"\n{name} Analysis:")
    for atom in mol.GetAtoms():
        print(f"Atom {atom.GetIdx()}: {atom.GetSymbol()} | Bonds: {atom.GetDegree()} | Valence: {atom.GetExplicitValence()}")

# Check each component
check_molecule("xxxxx.sdf", "WARHEAD")
check_molecule("RNF114.sdf", "BINDER")
check_molecule("CDK10.sdf", "LINKER")


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

# Function to load the first molecule from an SDF file.
def load_molecule(sdf_filename):
    supplier = Chem.SDMolSupplier(sdf_filename)
    if not supplier or len(supplier) == 0:
        raise IOError(f"Could not load any molecule from {sdf_filename}")
    mol = supplier[0]
    if mol is None:
        raise ValueError(f"Failed to parse molecule from {sdf_filename}")
    return mol

# Load molecules from SDF files.
warhead = load_molecule("RL559.sdf")
linker  = load_molecule("AR_Del_388-390.sdf")
binder  = load_molecule("cIAP1_0.sdf")

# Define a reaction SMARTS that connects two molecules at their dummy atoms.
# The reaction SMARTS "[*:1][*].[*:2][*]>>[*:1]-[*:2]" means:
#  - Remove a dummy atom from each reactant and form a single bond between the atoms formerly attached.
rxn = AllChem.ReactionFromSmarts("[*:1][*].[*:2][*]>>[*:1]-[*:2]")

# ---- Step 1: Connect Warhead and Linker ----
products = rxn.RunReactants((warhead, linker))
if products:
    # Take the first product as the intermediate.
    intermediate = products[0][0]
    # Skip kekulization if needed to avoid aromaticity issues.
    Chem.SanitizeMol(intermediate, sanitizeOps=Chem.SanitizeFlags.SANITIZE_ALL ^ Chem.SanitizeFlags.SANITIZE_KEKULIZE)
else:
    raise ValueError("Reaction between warhead and linker failed.")

# ---- Step 2: Connect the Intermediate and Binder ----
products_final = rxn.RunReactants((intermediate, binder))
if products_final:
    final_mol = products_final[0][0]
    Chem.SanitizeMol(final_mol, sanitizeOps=Chem.SanitizeFlags.SANITIZE_ALL ^ Chem.SanitizeFlags.SANITIZE_KEKULIZE)
else:
    raise ValueError("Reaction between intermediate and binder failed.")

# Output the final PROTAC molecule as a SMILES string.
final_smiles = Chem.MolToSmiles(final_mol)
print("Final PROTAC SMILES:", final_smiles)

# Optionally, write the final molecule to an SDF file.
writer = Chem.SDWriter("final_protac.sdf")
writer.write(final_mol)
writer.close()


In [ ]:
import os
import csv
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdchem

# === User parameters ===
binder_file   = "RNF114.sdf"
linker_file   = "CDK10.sdf"
warhead_dir   = "warheads"

# Heavy‐atom indices to join (0‐based)
binder_idx           = 0
linker_binder_idx    = 10
linker_warhead_idx   = 6
warhead_idx          = 29

# === Utility: load, embed, optimize, and cleanup Hs ===
def load_and_prep(path):
    mol = Chem.SDMolSupplier(path, removeHs=False)[0]
    if mol is None:
        raise IOError(f"Could not load '{path}'")
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    AllChem.UFFOptimizeMolecule(mol)
    # Remove explicit Hs to avoid valence issues
    mol = Chem.RemoveHs(mol)
    return mol

# 1) Load & prep binder + linker
binder = load_and_prep(binder_file)
linker = load_and_prep(linker_file)

# 2) Build base (binder + linker)
base_rw = Chem.RWMol(binder)
linker_map = {}
for atom in linker.GetAtoms():
    linker_map[atom.GetIdx()] = base_rw.AddAtom(atom)
for bond in linker.GetBonds():
    a1 = linker_map[bond.GetBeginAtomIdx()]
    a2 = linker_map[bond.GetEndAtomIdx()]
    base_rw.AddBond(a1, a2, bond.GetBondType())
# binder→linker bond
base_rw.AddBond(binder_idx, linker_map[linker_binder_idx], rdchem.BondType.SINGLE)

# Debug sanitize with detailed atom info if error
try:
    base_mol = base_rw.GetMol()
    Chem.SanitizeMol(base_mol)
except Chem.AtomValenceException as e:
    # Find offending atom(s)
    for atom in base_rw.GetMol().GetAtoms():
        if atom.GetExplicitValence() > Chem.GetPeriodicTable().GetDefaultValence(atom.GetAtomicNum()):
            print(f"Valence error at atom idx {atom.GetIdx()} ({atom.GetSymbol()}): "
                  f"explicit valence {atom.GetExplicitValence()} > allowed.")
    raise

# Prepare names & SMILES
binder_name  = os.path.splitext(os.path.basename(binder_file))[0]
linker_name  = os.path.splitext(os.path.basename(linker_file))[0]
binder_smile = Chem.MolToSmiles(binder)
linker_smile = Chem.MolToSmiles(linker)

# 3) Loop over warheads
rows = []
for fname in sorted(os.listdir(warhead_dir)):
    if not fname.lower().endswith(".sdf"):
        continue
    warpath = os.path.join(warhead_dir, fname)
    warname = os.path.splitext(fname)[0]

    try:
        # load & prep warhead
        war = load_and_prep(warpath)

        # combine base + warhead
        rw = Chem.RWMol(base_mol)
        war_map = {}
        for atom in war.GetAtoms():
            war_map[atom.GetIdx()] = rw.AddAtom(atom)
        for bond in war.GetBonds():
            a1 = war_map[bond.GetBeginAtomIdx()]
            a2 = war_map[bond.GetEndAtomIdx()]
            rw.AddBond(a1, a2, bond.GetBondType())
        # linker→warhead bond
        rw.AddBond(linker_map[linker_warhead_idx],
                   war_map[warhead_idx],
                   rdchem.BondType.SINGLE)

        # sanitize and optimize
        protac = rw.GetMol()
        Chem.SanitizeMol(protac)
        AllChem.EmbedMolecule(protac)
        AllChem.UFFOptimizeMolecule(protac)

        # write out SDF
        protac_name = f"{warname}_protac"
        sdf_out     = os.path.join(warhead_dir, f"{protac_name}.sdf")
        w = Chem.SDWriter(sdf_out)
        w.write(protac)
        w.close()

        # record CSV
        rows.append({
            "warhead_name":   warname,
            "warhead_smiles": Chem.MolToSmiles(war),
            "linker_name":    linker_name,
            "linker_smiles":  linker_smile,
            "binder_name":    binder_name,
            "binder_smiles":  binder_smile,
            "protac_name":    protac_name,
            "protac_smiles":  Chem.MolToSmiles(protac)
        })
        print(f"✅ Built PROTAC for {warname}")

    except Exception as e:
        print(f"⚠️ Skipping '{fname}' due to error: {e}")

# 4) Write summary CSV
csv_path = os.path.join(warhead_dir, "protac_summary.csv")
with open(csv_path, "w", newline="") as csvfile:
    fieldnames = [
        "warhead_name","warhead_smiles",
        "linker_name","linker_smiles",
        "binder_name","binder_smiles",
        "protac_name","protac_smiles"
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

print(f"\n✅ Finished. {len(rows)} PROTAC(s) generated. CSV summary at '{csv_path}'.")


In [ ]:
#!/usr/bin/env python
import os, glob, csv, re

LOG_PATTERN = "*.pdbqt_log.log"
OUTPUT_CSV  = "vina_mode1_scores.csv"

def parse_mode1_affinity(logpath):
    with open(logpath) as f:
        lines = f.readlines()
    # header‑based parse
    for i, L in enumerate(lines):
        if re.match(r"\s*mode\s*\|\s*affinity", L, re.IGNORECASE):
            idx = i + 2
            if idx < len(lines):
                parts = lines[idx].split()
                if parts and parts[0] == "1":
                    try:
                        return float(parts[1])
                    except:
                        pass
            break
    # fallback scan
    for L in lines:
        parts = L.split()
        if len(parts) >= 2 and parts[0] == "1":
            try:
                return float(parts[1])
            except:
                continue
    return None

def main():
    logs = glob.glob(LOG_PATTERN)
    records = []
    for log in logs:
        base = os.path.basename(log).split(".pdbqt")[0]
        score = parse_mode1_affinity(log)
        if score is None:
            print(f"[WARN] Could not parse mode‑1 from `{log}`")
        else:
            print(f"{base} → {score:.2f}")
        records.append((base, score))
    # write CSV
    with open(OUTPUT_CSV, "w", newline="") as outf:
        w = csv.writer(outf)
        w.writerow(["Compound", "Mode1_Affinity"])
        for comp, aff in records:
            w.writerow([comp, aff if aff is not None else ""])
    print(f"\nWritten {len(records)} entries to `{OUTPUT_CSV}`")

if __name__=="__main__":
    main()
